# 3.0 Water Occurrence

The intertidal grid that was generated in the previous step will now be used to produce a water occurrence analysis. The water occurrence functions will run for each individual cell, producing a seperate image for each cell.

## 3.1 Python Setup

We need to import the appropriate modules into python:

In [ ]:
# Standard library imports
import datetime
import time
import os

# Import Earth Engine
import ee

# Import Coast X-Ray Module
import cxrWaterOccurrence as cxr
cxr.ee = ee


Then, Google Earth Engine needs to be initialised (GEE needs to be authenticated before this step).

In [ ]:
# Initalise GEE
try:
  ee.Initialize()
  print('The Earth Engine package initialized successfully!')
except ee.EEException as e:
  print('The Earth Engine package failed to initialize!')
except:
    print("Unexpected error:", sys.exc_info()[0])
    raise

## 3.2 User inputs

The requires information from the user with regards GEE account inforomation, location of the intertidal grid (and its resolution), a AoI name, the start and end dates of the analysis, and also the location that the image collection should be placed on GEE.

In [ ]:
### USER INPUT STARTS ###

# inputs
#your GEE username
user = "username" # replace with username

# the path to the intertidal grid on GEE
intertidalGridPath = "GlobalGrid/AoI/ISEA3H_12_UKIre"

# the resolution of the grid
cxr.gridResolution = 12

# name of area of interest
aoiName = 'Area'

# start date for the analysis
cxr.startDate = '2015-09-01'

# end date for the analysis (automatically set as today's date)
cxr.endDate = str(datetime.date.today())
# cxr.endDate = '2019-09-30'
                 
# outputs
# the folder path you wish to export the water occurrence image collection to
exportFolderPath = "CoastXRayPython/Outputs"

# the folder path you wish to export the water occurrence feature collection to
exportFolderPathGridCellSummary = "CoastXRayPython/GridCellSummary"

# the folder path you wish to export the temporary water occurrence feature collection to
exportFolderPathGridCellSummaryTemp = "CoastXRayPython/GridCellSummaryTemp"

### USER INPUT ENDS ###


## 3.3 GEE Collection Directory

An empty image collection needs to be created on GEE in an appropriate folder that will recieve the output images:

In [ ]:
# Creates the folders and image collection on GEE to recieve the outputs

# Create the path
exportCollection = aoiName + '_' + cxr.startDate + '_' + cxr.endDate

# Create the export collection
os.system("earthengine create collection users/" + user + "/" + exportFolderPath + "/" + exportCollection + " -p")
os.system("earthengine create folder users/" + user + "/" + exportFolderPathGridCellSummary + "/" + exportCollection + " -p")
os.system("earthengine create folder users/" + user + "/" + exportFolderPathGridCellSummaryTemp + "/" + exportCollection + " -p")


## 3.4 Intertidal Grid and Sentinel 2 Image Collection

The intertidal grid needs to be accessed and then stored as a Feature Collection:

In [ ]:
# Construct the path to the intertidal grid
intertidalGrid = "users/" + user + "/" + intertidalGridPath
intertidalGrid = ee.FeatureCollection(intertidalGrid)
intertidalGridSize = intertidalGrid.size().getInfo()

print('The grid consists of ' + str(intertidalGridSize) + ' intertidal grid cells.')


The Sentinel 2 image collection is created then filtered firstly by cloud cover (images with cloud cover less then 90 are retained), then by time (using the start and end date set in 3.1), then by location (using the intertidal grid):

In [ ]:
# Create the inital Sentinel 2-1C collection
gridBounds = ee.Feature(intertidalGrid.union().first()).bounds().geometry()

collection = ee.ImageCollection("COPERNICUS/S2")\
    .filterMetadata('CLOUD_COVERAGE_ASSESSMENT', 'not_greater_than', 90)\
    .filterDate(cxr.startDate,cxr.endDate)\
    .filterBounds(gridBounds)

print('There are ' + str(collection.size().getInfo()) + ' images in the collection.')

## 3.5 Cloud Masking

Imagery is collected from the Sentinel 2 in all weather conditions. This means that images can be collected where the Earth's surface is partly, or even completely, obscured by clouds. Furthemore, cloud shadows can also create inaccuracies within the analysis, producing erroneous results. 

To limit these issues, firstly, images with extensive cloud cover are excluded from the image collection (see above). Secondly, clouds within the remaining Sentinel 2 images are removed (otherwise known as cloud masking). This is done using the Quality Assurrance band (QA60) in the image. The clouds are identified and then masked out of the image, and stored within the collectionCloudMasked image collection.

*Note that there are a number of ways to mask the cloud in an image - the approach used here is probably the simplest and could be improved in future versions*

In [ ]:
collectionCloudMasked = ee.ImageCollection(collection).map(cxr.cloudMask) 

## 3.6 Water Occurrence Analysis

The script below uses a for loop, which means that the script runs on each of the grid cells within the intertidal grid sequentially. On each grid cell the following steps are conducted:


In [ ]:
#Convert the feature collection to a list to allow looping:
intertidalGridList = intertidalGrid.toList(intertidalGridSize)

#an empty list to receive the GEE task ids
taskIDList = []

#an empty list to recieve the features of each cell
gridFeaturesList = []

#get a list of number to allow the looping through the feature collection
for i in range(intertidalGridSize):
# for i in range(1): #use this if you just want to test it on a single grid cell
    #get the grid cell feature
    cxr.gridCellFeature = ee.Feature(intertidalGridList.get(i))
    
    #get the geometry of the grid cell
    cxr.gridCell = ee.Feature(intertidalGridList.get(i)).buffer(20).geometry()
    
#STEP 1: Filter the collection to the boundary of the feature
    
    #SENTINEL2-1C
    cxr.gridCellCollection = collectionCloudMasked.filterBounds(cxr.gridCell)
    
#STEP 2: Clip the images in the collection to the extent of the grid cell
    #SENTINEL2-1C
    cxr.gridCellCollection = cxr.gridCellCollection.map(cxr.gridCellCollectionClip)

#STEP 3: Remove the duplicate images from the collection

    #SENTINEL2-1C
    cxr.gridCellCollection = cxr.gridCellCollection.map(cxr.removeDuplicates)
    cxr.gridCellCollection = ee.ImageCollection(cxr.gridCellCollection).distinct('dateId')

#STEP 4: Mosaic the images that occur on the same day/time    

    #SENTINEL2-1C
    cxr.gridCellCollection = cxr.mosaicSameDay(ee.ImageCollection(cxr.gridCellCollection))
        
#STEP 5: Calculate the area of the image within the grid cell and set it as a property of the image

    #SENTINEL2-1C   
    cxr.gridCellCollection = cxr.gridCellCollection.map(cxr.setImageArea)

#STEP 6: Filter out the images that do not sufficiently cover the grid cell

    #the cloud cover filter value (%) to start 
    startFilter = 99 #up to 99.5?
    
    #the interval to reduce the cloud cover filter by each iteration
    interval = 0.5
    
    #number of filters to test
    iterations = 5
    
    #the minimum mumber of images required to make the water occurrence output - default = 30
    minImages = 30
    
    #the default cloud cover filter value (%) if a better one cannot be found
    defaultFilter = 90
    
    #find the cloud cover value
    cxr.cellCloudCoverFilterValue = cxr.setFilterDecending(cxr.gridCellCollection, startFilter, interval, iterations, minImages, defaultFilter)
    
#STEP 7: Filter the collection based on the cloud cover value
    cxr.gridCellCollection = cxr.gridCellCollection.filterMetadata('gridCellCoverage', 'greater_than', cxr.cellCloudCoverFilterValue)

#STEP 8: Convert the images in the collection into NDWI images and identify the water in each image
    
    #calulate the NDWI for each image
    gridCellNDWICollection = ee.ImageCollection(cxr.gridCellCollection).map(cxr.NDWI)
    
    #set the threshold for water extraction from the NDWI image
    cxr.ndwiThreshold = 0.2

    gridCellNDWIWaterCollection = gridCellNDWICollection.map(cxr.ndwiWater)
    
#STEP 9: Calculate the water occurrence of the collection

    #count the numer of water occurrences at each pixel
    waterReduceSum = gridCellNDWIWaterCollection.reduce(ee.Reducer.sum()).int16().rename('waterOccurrenceCount')
    
    #remove pixels that only have 1 for water occurence - QA check - review this
    waterReduceSum = waterReduceSum.where(waterReduceSum.eq(1), 0)

    #calculates the water occurence % 
    cxr.gridCellNDWICollectionSize = gridCellNDWICollection.size()
    waterPercentage = waterReduceSum.divide(cxr.gridCellNDWICollectionSize).multiply(100).rename('waterOccurrencePercentage').addBands(waterReduceSum)

    #adds the image collection size as a band
    mask = waterPercentage.select('waterOccurrenceCount').mask()
    
    #add a band within the water occurrence image that holds the number of images used to calculate the water occurrence
    bandCollectionLength = ee.Image.constant(cxr.gridCellNDWICollectionSize).uint16().rename('numberOfImagesAnalysed').updateMask(mask)
    gridCellWaterOccurrenceOutput = waterPercentage.addBands(bandCollectionLength)
    
#STEP 10: Create a median NDWI band and add the band to the water occurrence image
    
    #calculate the median NDWI
    ndwiMedian = gridCellNDWICollection.median().rename('ndwiMedian')
    
    #add ndwiMedian as a band to the water occurrence image
    gridCellWaterOccurrenceOutput = gridCellWaterOccurrenceOutput.addBands(ndwiMedian)
    
# #STEP 11: Create a feature collection, with each feature containing information about the images used
    
    #get the grid cell number and convert it to string
    cxr.cellNumber = cxr.gridCellFeature.get('cell').getInfo()    
    cxr.cellNumberStr = str(cxr.cellNumber)
    
    #get the date of the earliest image 
    cxr.earliestImage = ee.Date(cxr.gridCellCollection.sort('system:time_start', True).first().get('dateTime')).format("YYYY-MM-dd")
    
    #get the date of the earliest image 
    cxr.latestImage = ee.Date(cxr.gridCellCollection.sort('system:time_start', False).first().get('dateTime')).format("YYYY-MM-dd")
    
    #add the image metadata to the feature properties
    gridCellWaterOccurrenceOutput = gridCellWaterOccurrenceOutput\
    .set('numberOfImagesAnalysed', ee.Number(cxr.gridCellNDWICollectionSize))\
    .set('analysisStart', ee.Date(cxr.startDate).format("YYYY-MM-dd"))\
    .set('analysisEnd', ee.Date(cxr.endDate).format("YYYY-MM-dd"))\
    .set('earliestImageAnalysed', cxr.earliestImage)\
    .set('latestImageAnalysed', cxr.latestImage)\
    .set('cellCloudCoverThreshold', ee.Number(cxr.cellCloudCoverFilterValue))\
    .set('cell', ee.Number(cxr.cellNumber))\
    .set('cellResolution', ee.Number(cxr.gridResolution))\
      
#STEP 12 - Add the grid cell feature to a collection of the other grid cell features 

    #get the list of dates
    dateList = cxr.gridCellCollection.aggregate_array('dateTime')

    #append the cell geometry and properties to the main list
    gridCellFeatures = ee.List(dateList).map(cxr.featureProperties)
    
#STEP 13: Export images

    #EXPORT
    #generate the export file name
    exportFileName =  str(cxr.gridResolution) + "_" + cxr.cellNumberStr + "_" + aoiName + "_" + cxr.startDate + "_" + cxr.endDate
    exportPath = "users/" + user + "/" + exportFolderPath + "/" + exportCollection + "/" + exportFileName 
    exportPathGrid = "users/" + user + "/" + exportFolderPathGridCellSummaryTemp +  "/" + exportCollection + "/" + exportFileName + '_GCS'
    
    #create the export task and start it   
    task = ee.batch.Export.image.toAsset(**{
        'image': gridCellWaterOccurrenceOutput,
        'description': "CoastXRay: " + cxr.cellNumberStr, 
        'assetId': exportPath,
        'region': cxr.gridCell.getInfo()['coordinates'],
        'scale': 10
        })
    task.start()
    print('Started:' + exportFileName)
    
    #get the task ID and append it to the list
    taskID=task.status()['id']
    taskIDList.append(taskID)

    task = ee.batch.Export.table.toAsset(**{
            'collection': ee.FeatureCollection(gridCellFeatures).sort('dateMilli'), 
            'description': "CoastXRay CGS: " + cxr.cellNumberStr,
            'assetId': exportPathGrid
            })

    task.start()
    print('Started:' + exportFileName + '_GCS')

## 3.7 Merge Grid Cell Summary Features

Each of the grid cells have a feature collection produced which summaries the images used. To make this easier to manage it is best to merge the all the feature collections in to one master dataset. 

**This should be run after all the cells have been processed on GEE.**


In [ ]:
# merge all the grid cell summary features into one feature collection

# get a list of the features in the temporary directory
assetList = ee.data.getList({'id': exportFolderPathGridCellSummaryTemp})

# count the size and create a list range 
assetListSize = ee.List(assetList).size().getInfo()
listRange = list(range(0, assetListSize-1))

# create an empty feature collection
fc = ee.FeatureCollection([])

# get all the features and merge them into one feature collection
for i in listRange:
    fc = fc.merge(ee.FeatureCollection(assetList[i]['id']))  

# export the feature collection
task = ee.batch.Export.table.toAsset(**{
            'collection': fc, 
            'description': "Grid Cell Summary Feature Collection",
            'assetId': "users/" + user + "/" + exportFolderPathGridCellSummary + "/" + exportCollection
            })

task.start()

**END**